In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [2]:
tracks = pd.read_csv('../../Dataset/data_processed.csv')

In [3]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381292 entries, 0 to 381291
Data columns (total 28 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   popularity        381292 non-null  int64  
 1   explicit          381292 non-null  int64  
 2   release_date      381292 non-null  int64  
 3   danceability      381292 non-null  float64
 4   energy            381292 non-null  float64
 5   loudness          381292 non-null  float64
 6   speechiness       381292 non-null  float64
 7   acousticness      381292 non-null  float64
 8   instrumentalness  381292 non-null  float64
 9   liveness          381292 non-null  float64
 10  valence           381292 non-null  float64
 11  tempo             381292 non-null  float64
 12  time_signature    381292 non-null  int64  
 13  key_0             381292 non-null  int64  
 14  key_1             381292 non-null  int64  
 15  key_2             381292 non-null  int64  
 16  key_3             38

In [4]:
tracks.popularity.describe()
mean_popularity = 50
tracks["popularity"] = [ 1 if i >= mean_popularity else 0 for i in tracks.popularity ]
tracks.head()

,popularity,explicit,release_date,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,...,key_5,key_6,key_7,key_8,key_9,key_10,key_11,minor,major,duration
0,1,0,2008,0.484,0.265,-11.101,0.0322,0.394,0.000000,0.149,...,0,0,0,0,0,0,0,0,1,160
1,0,0,2020,0.671,0.867,-2.706,0.0571,0.436,0.000000,0.139,...,0,0,0,0,0,0,0,0,1,160
2,0,0,2008,0.405,0.365,-10.226,0.0289,0.255,0.000005,0.163,...,0,1,0,0,0,0,0,1,0,142
3,0,0,2008,0.477,0.352,-14.165,0.0300,0.406,0.000000,0.122,...,0,0,0,0,0,0,0,0,1,157
4,0,0,2018,0.319,0.201,-17.796,0.0623,0.887,0.000000,0.904,...,0,0,1,0,0,0,0,0,1,187


In [5]:
#tracks['popularity'] = pd.qcut(tracks['popularity'], q=2, labels=[0, 1])

X  = tracks.drop(['popularity', 'release_date'], axis=1)
y = tracks['popularity']

In [6]:
%%time

from sklearn.feature_selection import SelectPercentile, chi2

input_scaler = StandardScaler()
X_normalized = input_scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.20, random_state=42, stratify=y
)

CPU times: total: 703 ms
Wall time: 703 ms


In [7]:
X_train.shape, X_test.shape

((305033, 26), (76259, 26))

In [8]:
%%time
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from joblib import dump


svm = SVC(kernel='linear',
          C=1000,
          random_state=42)
svm.fit(X_train[:10000], y_train[:10000])
dump(svm, "mlspotify_SVM")

print("Train accuracy of svm algo:", svm.score(X_train[:10000], y_train[:10000]))
print("Test accuracy of svm algo:", svm.score(X_test, y_test))



<timed exec>:10: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.


In [ ]:
y_pred = svm.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print("Classification report\n")
print(classification_report(y_test, y_pred))
     